In [4]:
import mediapipe as mp
mp_holistic = mp.solutions.holistic
# Import drawing_utils and drawing_styles.
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

In [5]:
import cv2
import math
import numpy as np

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
    cv2.imshow("resizedimage", image)
    #waits for user to press any key 
    #(this is necessary to avoid Python kernel form crashing)
    cv2.waitKey(0) 
    #closing all open windows 
    cv2.destroyAllWindows()

In [6]:
#capture the video, and check video settings
capture = cv2.VideoCapture('.//Video//sample.mp4')
frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = capture.get(cv2.CAP_PROP_FPS)   #fps = frames per second
print(frameWidth, frameHeight, fps)
#make an 'empty' video file where we project the poste tracking on
samplerate = fps #make the same as current video
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #(*'XVID')
out = cv2.VideoWriter('AnonymizedVideos/'+'sample_anonymized'+'.mp4', fourcc, fps = samplerate, frameSize = (int(frameWidth), int(frameHeight)))

# Run MediaPipe Holistic with `enable_segmentation=True` to get pose segmentation.
with mp_holistic.Holistic(
        static_image_mode=True, enable_segmentation=True, refine_face_landmarks=True) as holistic:
    while (True):
        ret, image = capture.read() #read frames
        if ret == True:
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            # Draw pose segmentation.
            annotated_image = image.copy()  
            h, w, c = annotated_image.shape
            original_image = np.concatenate([annotated_image, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
            red_img = np.zeros_like(annotated_image, dtype=np.uint8)
            red_img[:, :] = (255,255,255)
            segm_2class = 0.2 + 0.8 * results.segmentation_mask
            segm_2class = np.repeat(segm_2class[..., np.newaxis], 3, axis=2)
            annotated_image = annotated_image * segm_2class * (1 - segm_2class)
            annotated_image2 = annotated_image * segm_2class * (1 - segm_2class)
                    # get the image dimensions (height, width and channels)
            # append Alpha channel -- required for BGRA (Blue, Green, Red, Alpha)
            mask = np.concatenate([annotated_image, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
            # Zero background where we want to overlay
            original_image[mask==0]=0
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
            mp_drawing.draw_landmarks(original_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(original_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(
                    original_image,
                    results.face_landmarks,
                    mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(
                    original_image,
                    results.pose_landmarks,
                    mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.
                    get_default_pose_landmarks_style())
            cv2.imshow("resizedimage", original_image)#original_image)
            out.write(original_image)
            #waits for user to press any key 
            #(this is necessary to avoid Python kernel form crashing)
            #cv2.waitKey(0)
            if cv2.waitKey(1) == 27: #allow the use of ESCAPE to break the loop
                break
        if ret == False: #if there are no more frames, break the loop
            break
            
#once done de-initialize
out.release()
capture.release()
cv2.destroyAllWindows()

1280.0 720.0 30.00115834588208


In [3]:
#extract audio from file and save it in Audio folder
import moviepy.editor as mp
my_clip = mp.VideoFileClip(".//Video//sample.mp4")
my_clip.audio.write_audiofile(".//Audio//sample.wav")

MoviePy - Writing audio in .//Audio//sample.wav


MoviePy - Done.


In [13]:
#go through audio, chunk it, and extract text
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
import pyttsx3 

# a function that splits the audio file into chunks
# and applies speech recognition https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  

    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened, language="en-EN") # "nl-NL", "es-ES", "fr-FR", "it-IT"
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text
    

# Initialize the recognizer 
r = sr.Recognizer() 

speechtotext =  get_large_audio_transcription(".//Audio//sample.wav")
#write transcriptions; note please check transcriptions to correct if neccesary
with open('.//Transcriptions//sample.txt', 'w') as f:
    f.write(speechtotext)

audio-chunks\chunk1.wav : Is the question i don't even get no to ask. 
audio-chunks\chunk2.wav : The questionnaire will only arrive after you. 
audio-chunks\chunk3.wav : Work your way to the edge and you find yourself standing on a new vista and staring out into the unknown and say i have a new question. 
audio-chunks\chunk4.wav : About what's going on out there. 


In [10]:
import sys
import autosub

autosub(# pylint: disable=too-many-locals,too-many-arguments
        ".//Audio//sample.wav",
        output=".//subtitles//sample.srt",
        src_language='en',
        dst_language='en',
        subtitle_file_format='srt',
        api_key=None,
    )

TypeError: 'module' object is not callable

TypeError: 'module' object is not callable

In [2]:
##force aligner throws error
#from pydub.utils import mediainfo
#import numpy
#import pyfoal
#from scipy.io.wavfile import read as read_wav
#
#text_file = ".//Transcriptions//sample.txt"
#audio_file = ".//Audio//sample.wav"
#sampling_rate, data=read_wav(audio_file) # enter your filename
#
##read text
#with open(text_file) as f:
#    lines = f.readlines()
#print(lines)
##load audio into 1D array
#audio = numpy.array(data,dtype=float)
#print(audio)
#
#alignment = pyfoal.align(text, audio, sample_rate=sampling_rate)